# Problem 1 (Practice)

We learned how to construct an AND operator in the lecture. We will repeat this exercise, this time constructing an OR operator.

(a) Taking the two inputs as $x_1$ and $x_2$, predict what the values of the fitting parameters $W_1, W_2,$ and $b$ should be, where $v = W_1x_1 + W_2x_2 + b$. (10 pts)

$W_1 = 1, W_2 = 1, b = -0.5$

(b) Write a program that learns to make the OR operator using perceptrons. Use $Nep=1000$ epochs, and a learning rate of $c=0.5$, and the sigmoid activation function
\begin{align*}
  y=h(v)=\frac{1}{1+e^{-v}}.
\end{align*}
Use the gradient descent method as the learning algorithm. Use the initial guesses $W_1 = 1, W_2=0.1, b=-1$. For every epochs, save the squared error, $\mathbf{W}$, and $b$. Plot the squared error as a function of epoch. Print out the initial and final $\mathbf{W}$ vector and bias $b$. Do the final values match your predictions? [Hint: Normalize $\mathbf{W}$ and $b$ by $W_1$ at when printing them.] (40 pts)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 4
Nep = 1000
c = 0.5

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x_train = np.array([[1,1], [1,0], [0,1], [0,0]])
a_train = np.array([1, 1, 1, 0])

e = np.zeros(4)
W = np.array([1,0.1])
b = -1

xarr = []
yarr = []
Warr = []
barr = []
print(W/W[0], b/W[0])

for ep in range(Nep):
    for n in range(N):
        x = x_train[n]
        a = a_train[n]
        v = np.sum(W * x) + b
        y = sigmoid(v)
        e = a - y
        W = W + c*e*y*(1-y)*x
        b = b + c*e*y*(1-y)
    xarr.append(ep)
    yarr.append(e*e)
    Warr.append(W)
    barr.append(b)

print(W/W[0], b/W[0])
plt.plot(xarr, yarr)
plt.xlabel("Epoch")
plt.ylabel("Error")
plt.show()


(c) Generate 1000 random $(x_1,x_2)$ points from (0,0) to (1,1). Make a scatter plot of these points, with the colors representing the value of $h(v)$. Are the points well-classified? (15 pts)

In [ ]:
xlist = []
ylist = []

for i in range(1000):
    x = np.random.random(2)
    y = np.sum(W@x) + b
    xlist.append(x)
    ylist.append(sigmoid(y))

colors = plt.cm.RdBu(ylist)
xlist = np.array(xlist)
plt.scatter(xlist[:,0], xlist[:,1], marker='o',c=colors)
plt.xlabel("x1")
plt.ylabel("x2")
plt.xlim(0,1)
plt.ylim(0,1)
ax = plt.gca()
ax.set_aspect('equal')
plt.show()

(d) Make a plot of the separator line $v=0$ as a function of epochs. (15 pts)

In [ ]:
x1 = np.linspace(0, 1, 100)
colors = plt.cm.viridis(np.linspace(0, 1, len(Warr)))
for n in range(len(barr)):
    plt.plot(x1, -(Warr[n][0]*x1+barr[n])/Warr[n][1], c=colors[n])
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim(0, 1)
plt.ylim(0, 1)
ax = plt.gca()
ax.set_aspect('equal')
plt.show()

(e) The bias $b$ can actually by written as $W_0$, and then $v=\sum_{n=0}^2 W_n x_n$ where $x_0=1$. Modify your code so that instead of $b$, you provide a training data set including $x_0=1$ and learn the weight vector with three elements. Print out the final weight vector. Do the values agree with your previous result? (20 pts)

In [ ]:
x_train = np.array([[1,1,1], [1,1,0], [1,0,1], [1,0,0]])
a_train = np.array([1, 1, 1 ,0])
e = np.zeros(4)
W = np.array([-1,1,0.1])
xarr = []
yarr = []
Warr = []
print(W/W[1])
for ep in range(Nep):
    for n in range(N):
        x = x_train[n]
        a = a_train[n]
        v = np.sum(W*x)
        y = sigmoid(v)
        e = a - y
        W = W + c*e*y*(1-y)*x
        b = b + c*e*y*(1-y)
    xarr.append(ep)
    yarr.append(e*e)
    Warr.append(W)

print(W/W[1])

---
# Problem 2

In the lecture, the updates for the weight vector elements were shown according to the gradient descent method, namely
\begin{align*}
    \Delta W_{ij} &= - c' \frac{\partial E}{\partial W_{ij}} = c e_i y_i (1-y_i)x_j,\\
    \Delta b_i &= -c' \frac{\partial E}{\partial b_i}=c e_i y_i(1-y_i),
\end{align*}
where $c'$ is the learning rate.

Use the error function
\begin{align*}
    E= \sum_i (a_i-y_i)^2,
\end{align*}
where $a_i$ is the training data, to derive the updates for the sigmoid activation function
\begin{align*}
    v_i &= \sum_j W_{ij}x_j + b_i,\\
    y_i &= h(v_i).
\end{align*}

(30 pts)

---
# Problem 3

In this problem, we will see that the gradient descent method is not the only learning algorithm, of which there can be various types.

Consider the activation function
\begin{align*}
    y = h(v) = \text{sign}\left({\sum_i W_{i}x_i }\right).
\end{align*}
This is similar but different from the Heaviside step function in that False is represented by -1 instead of 0.

Instead of the gradient descent method, we will use the following algorithm:
1.  Identify a misclassified point $y_n$ in the training data, i.e.,$\text{sign}\left(\sum_i W_{i}x_{in}\right) = y_n = -a_n$ where $a_n$ is the training output.

2.  Update the weight vector so that $W_{i} \rightarrow W_{i} + a_n x_{in}$

3.  Repeat until all training points are classified correctly.

(a) Explain why this algorithm would update the weight vector toward the correct value. [Hint: If a misclassified $y_n=1$, that means $a_n=-1$ and so the change in $W_i$ should act in such a way that decreases the argument of $h(v)$. Similarly for a misclassified $y_n=-1$.] (20 pts)

(b) Implement this algorithm for the operator for the initial weight vector $\mathbf{W}=(-2,1,0.2)$. What is the final weight? How many epochs did it take? (40 pts)

(c) Repeat Problem 1(c) for the obtained weight. Are the training points classified correctly? [Hint: there are an infinite number of lines that can separate the four points according to the OR operator output] (10 pts)